<img src='https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png' alt="SUT logo" width=300 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Machine Learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2024<br>
<font color=3C99D size=5>
    Practical Assignment 5 - PCA  <br>
<font color=696880 size=4>
Parsa Sharifi

Full Name : Radin Cheraghi

Student Number : 401105815

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file.

In [1]:
# you can use the following packages feel free to add other libraries
import zipfile
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe

Flatten and normalize the images
Note: For this part, normalization only means dividing by 255.

In [2]:
zip_path = "images.zip"  # Replace with the path to your zip file
current_path = os.getcwd()  # Get the current working directory
extract_path = os.path.join(current_path, "content")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(current_path)

image_folder = os.path.join(extract_path, "img-data")
metadata_path = "faces-metad.csv"

metadata = pd.read_csv(metadata_path)

In [3]:
image_list = []
labels = []

for idx, row in metadata.iterrows():
    image_file = row["path"]
    label = row["label"]
    image_path = os.path.join(image_folder, image_file)
    try:
        # Load image and convert to grayscale
        image = Image.open(image_path).convert("L")
        # Resize to a fixed size (e.g., 32 * 32)
        image = image.resize((32, 32))
        # Convert to numpy array and normalize
        image_array = np.array(image) / 255.0
        image_list.append(image_array.flatten())  # Flatten the image
        labels.append(label)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

x = np.array(image_list)
y = np.array(labels)

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y`

In [4]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
metadata

,path,label
0,0.jpg,jones
1,1.jpg,jones
2,2.jpg,jones
3,3.jpg,jones
4,4.jpg,jones
...,...,...
395,395.jpg,vaux
396,396.jpg,vaux
397,397.jpg,vaux
398,398.jpg,vaux


Split the data into training (80%), validation (10%), and testing (10%) parts.

In [5]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Sec 2: PCA


## Subsec 2.1.

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`.

In [9]:
labels_to_fetch = ["jones", "taylor", "anderson", "wilson"]
encoded_labels_to_fetch = label_encoder.transform(labels_to_fetch)

filtered_images = []
for i, label in enumerate(y):
    if label in encoded_labels_to_fetch:
        filtered_images.append(x[i])

filtered_images = np.array(filtered_images)
print(f"Filtered images shape: {filtered_images.shape}")

class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean

        covariance_matrix = np.cov(X_centered, rowvar=False)

        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]

        self.components = eigenvectors[:, :self.n_components]

    def transform(self, X):
        X_centered = X - self.mean
        return np.dot(X_centered, self.components)

pca = PCA(n_components=3)
pca.fit(filtered_images)
reduced_data = pca.transform(filtered_images)

print(f"Reduced data shape: {reduced_data.shape}")

np.save("reduced_data.npy", reduced_data)
print("Saved reduced data as 'reduced_data.npy'")

Filtered images shape: (40, 1024)
Reduced data shape: (40, 3)
Saved reduced data as 'reduced_data.npy'


Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color.


In [10]:
reduced_data = np.real(reduced_data)

df = pd.DataFrame(reduced_data, columns=["PC1", "PC2", "PC3"])

df["label"] = [label_encoder.inverse_transform([label])[0] for label in encoded_labels_to_fetch for _ in range(10)]

fig = px.scatter_3d(
    df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="label",
    title="3D PCA Visualization of Selected Samples",
    labels={"label": "Person"}
)

fig.show()

What do you find out from the plot? write your analysis about the data

The 3D PCA plot shows distinct clusters for each person, indicating PCA effectively captured the key variations in facial features. Most clusters are well-separated, though there is slight overlap, especially between anderson and wilson, suggesting some similarity in their features. This dataset is suitable for classification tasks, and PCA successfully reduces dimensionality while preserving separability. Further analysis could focus on minimizing overlaps or using supervised techniques for better isolation.